In [4]:
# this notebook is to train bot trading stocking with reinforcement learning

# import helper functions on getting stock data
from getstock import get_stock_data_yf

# import library for gym, stable-baselines3, A2C, and numpy
import gym
import gym_anytrading
import numpy as np
from matplotlib import pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C


In [5]:
# get stock data on Apple for the past 1 year with 5 days intervals
stock_data = get_stock_data_yf('AAPL', 1, '1d')

# show the first 5 rows of the stock data
stock_data.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-26,149.330002,150.839996,149.009995,149.320007,148.487946,60893400
2021-10-27,149.360001,149.729996,148.490005,148.850006,148.020569,56094900
2021-10-28,149.820007,153.169998,149.720001,152.570007,151.719833,100077900
2021-10-29,147.220001,149.940002,146.410004,149.800003,148.965271,124953200
2021-11-01,148.990005,149.699997,147.800003,148.960007,148.129959,74588300


In [14]:
# create a gym environment for trading stock with the stock data
env = gym.make('stocks-v0', df=stock_data, frame_bound=(5, 100), window_size=5)

# check the environment's space
print("The environment's action space is: ", env.action_space.n)
print("The environment's observation space is: ", env.observation_space.shape[0])

# Sample a random action from the environment's action space and observation space
print("Sample action: ", env.action_space.sample())
print("Sample observation: ", env.observation_space.sample())

The environment's action space is:  2
The environment's observation space is:  5
Sample action:  0
Sample observation:  [[ 0.07905011  1.02224912]
 [ 0.998537   -0.10207556]
 [ 0.03110137 -1.31467876]
 [-1.14713899 -1.22494376]
 [-0.04354117  0.540746  ]]
